<a href="https://colab.research.google.com/github/DIPANJAN001/Andrew-Ng-Machine-Learning-Notes/blob/master/hackathon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Lambda
from tensorflow.keras.optimizers import Adam

# Load your dataset
data = pd.read_csv('your_dataset.csv')

# Split data into features (categorical and text) and labels
X_categorical = data[['claim_country', 'claim_state', 'line_of_business', 'litigation']]
X_text = data['loss_description']
y = data['severity']

# Preprocess the text data as mentioned earlier (tokenization, padding, etc.)

# Preprocess the categorical data (one-hot encoding or label encoding)

# Split data into training, validation, and test sets
X_categorical_train, X_categorical_test, X_text_train, X_text_test, y_train, y_test = train_test_split(X_categorical, X_text, y, test_size=0.2, random_state=42)

# Siamese network architecture

# Create shared layers for processing text and categorical data
embedding_dim = 64
text_embedding_layer = Embedding(input_dim=vocab_size, output_dim=embedding_dim)
categorical_embedding_layer = Embedding(input_dim=num_categories, output_dim=embedding_dim)

text_branch = LSTM(64)
categorical_branch = Flatten()

# Inputs
input_text1 = Input(shape=(max_text_sequence_length,))
input_text2 = Input(shape=(max_text_sequence_length,))
input_categorical1 = Input(shape=(X_categorical_train.shape[1],))
input_categorical2 = Input(shape=(X_categorical_train.shape[1],))

# Process text data
text_embedding1 = text_embedding_layer(input_text1)
text_embedding2 = text_embedding_layer(input_text2)
text_output1 = text_branch(text_embedding1)
text_output2 = text_branch(text_embedding2)

# Process categorical data
categorical_embedding1 = categorical_embedding_layer(input_categorical1)
categorical_embedding2 = categorical_embedding_layer(input_categorical2)
categorical_output1 = categorical_branch(categorical_embedding1)
categorical_output2 = categorical_branch(categorical_embedding2)

# Merge branches
merged = Lambda(lambda x: abs(x[0] - x[1]))([text_output1, text_output2])
merged = Concatenate()([merged, categorical_output1, categorical_output2])

# Fully connected layers for classification
dense_layer = Dense(64, activation='relu')(merged)
output_layer = Dense(1, activation='sigmoid')(dense_layer)

# Create the Siamese model
model = Model(inputs=[input_text1, input_text2, input_categorical1, input_categorical2], outputs=output_layer)

# Compile the model
model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])

# Train the Siamese model
# You will need to prepare pairs of examples for training
# For example, pair each claim with a random other claim (non-severe or severe)
# You would need to create a custom training loop for this setup
# Make sure that each batch contains positive and negative claim pairs

# Evaluate the model on your test set


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Lambda, Concatenate, Flatten
from tensorflow.keras.optimizers import Adam

# Load your dataset
data = pd.read_csv('your_dataset.csv')

# Preprocess text data
max_text_words = 10000  # Maximum number of words to tokenize
max_text_sequence_length = 100  # Maximum sequence length for padding
tokenizer = Tokenizer(num_words=max_text_words)
tokenizer.fit_on_texts(data['loss_description'])
text_sequences = tokenizer.texts_to_sequences(data['loss_description'])
X_text = pad_sequences(text_sequences, maxlen=max_text_sequence_length)

# Preprocess categorical data
categorical_cols = ['claim_country', 'claim_state', 'line_of_business', 'litigation']
for col in categorical_cols:
    data[col] = data[col].astype('category').cat.codes  # Convert to numerical labels

X_categorical = data[categorical_cols].values

# Target variable
y = data['severity'].values

# Split data into training and test sets
X_text_train, X_text_test, X_categorical_train, X_categorical_test, y_train, y_test = train_test_split(
    X_text, X_categorical, y, test_size=0.2, random_state=42
)

# Siamese network architecture

# Text branch
text_input = Input(shape=(max_text_sequence_length,))
text_embedding_layer = Embedding(input_dim=max_text_words, output_dim=64)(text_input)
text_lstm = LSTM(64)(text_embedding_layer)

# Categorical branch
categorical_input = Input(shape=(X_categorical_train.shape[1],))
categorical_embedding_layer = Embedding(input_dim=np.max(X_categorical_train) + 1, output_dim=32)(categorical_input)
categorical_flatten = Flatten()(categorical_embedding_layer)

# Merge branches
merged = Concatenate()([text_lstm, categorical_flatten])

# Fully connected layers for classification
dense_layer = Dense(64, activation='relu')(merged)
output_layer = Dense(1, activation='sigmoid')(dense_layer)

# Create the Siamese model
model = Model(inputs=[text_input, categorical_input], outputs=output_layer)

# Compile the model
model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])

# Train the model
model.fit([X_text_train, X_categorical_train], y_train, batch_size=32, epochs=10, validation_split=0.1)

# Evaluate the model
accuracy = model.evaluate([X_text_test, X_categorical_test], y_test)
print("Test Accuracy:", accuracy[1])

# Get probabilities for new claims
new_claim_text = tokenizer.texts_to_sequences(["Your new claim description"])
new_claim_text = pad_sequences(new_claim_text, maxlen=max_text_sequence_length)
new_claim_categorical = np.array([[claim_country_code, claim_state_code, line_of_business_code, litigation_code]])  # Replace with actual codes
probability_severe = model.predict([new_claim_text, new_claim_categorical])
print("Probability of being severe:", probability_severe)
